# Install Syncthing

* Purpose: synchronize directories between computers on local network
    * I use this to synchronize folders between my Mac and the Pi
* [Official Site](https://syncthing.net/)
* [Install Guide](https://pimylifeup.com/raspberry-pi-syncthing/)

## Install Software

In [ ]:
%%bash

sudo apt update
sudo apt-get -y full-upgrade
sudo apt-get -y install apt-transport-https
curl -s https://syncthing.net/release-key.txt | sudo apt-key add -
echo "deb https://apt.syncthing.net/ syncthing stable" | sudo tee /etc/apt/sources.list.d/syncthing.list
sudo apt update
sudo apt install syncthing 

## Ignore Files

1. Discussion: https://forum.syncthing.net/t/useful-stignore-patterns/1175
2. Gobal ignore file: https://gist.github.com/marksharrison

#### `.stglobalignore`

In [ ]:
%%bash

# list of filed/dirs that never sync (https://docs.syncthing.net/users/ignoring.html)
cat << 'EOF' > $IOT49/.stglobalignore
// .stglobalignore
// Syncthing Ignore Patterns - https://docs.syncthing.net/users/ignoring.html
// #patterns:
// Useful .stignore Patterns - https://goo.gl/DVHI1h

// Usage: Add the line below to all .stignore files for each Syncthing node
// #include .stglobalignore

// Incomplete Downloads
// Firefox downloads
*.part
// Chrom(ium|e) downloads
*.crdownload
// Transmission downloads
*.part

// Temporary/Backup Files
*~
.*.swp

// OS-generated files (linux)
.directory
.Trash-*

// OS-generated files (macOS)
ehthumbs.db
Thumbs.db.DS_Store
.AppleDouble
.LSOverride
// Icon must end with two \r
Icon
Icon?
// Thumbnails (metadata)
._*
// Files that might appear in the root of a volume
.DocumentRevisions-V100
.fseventsd
.Spotlight-V100
.TemporaryItems
.Trashes
.VolumeIcon.icns
.com.apple.timemachine.donotpresent
.localized
// Directories potentially created on remote AFP share
.AppleDB
.AppleDesktop
Network Trash Folder
Temporary Items
.apdisk
// iCloud temp files
.iCloud*

// OS-generated files (Windows)
// Windows thumbnail cache files
Thumbs.db
Thumbs.db:encryptable
ehthumbs.db
ehthumbs_vista.db
// Dump file
*.stackdump
// Folder config file
[Dd]esktop.ini
// Recycle Bin used on file shares
$RECYCLE.BIN/
// Windows Installer files
*.cab
*.msi
*.msix
*.msm
*.msp
// Windows shortcuts
*.ln
// Microsoft Office temp files
(?d)~*

// BTSync files
.sync
*.bts
*.!Sync
.SyncID
.SyncIgnore
.SyncArchive
*.SyncPart
*.SyncTemp
*.SyncOld

// Synology files
@eaDir

// Syncthing files
.stignore
.stfolder

// vim swap files
(?d)*.*.sw[a-p]

// Jupyter
.ipynb_checkpoints/
.ipynb_checkpoints*

// Virtual Env
.envrc
.direnv
.git

// Misc
*tmp
__pycache__
*.py[cod]
*$py.class
*bak

EOF

#### `.stignore`

In [ ]:
%%bash 

# list of filed/dirs that never sync (https://docs.syncthing.net/users/ignoring.html)
cat << 'EOF' > $IOT49/.stignore
#include .stglobalignore

EOF

## Configure software

In [ ]:
%%bash

# start then kill to create default configuration
syncthing &
last_pid=$!
echo $last_pid
sleep 5
kill -KILL $last_pid

# web access from any authorized device
cd ~/.config/syncthing
cp config.xml config.xml.bak
sed -i 's/127\.0\.0\.1:8384/0\.0\.0\.0:8384/g' config.xml

# install service
sudo tee /lib/systemd/system/syncthing.service << EOF
[Unit]
Description=Syncthing - Open Source Continuous File Synchronization
Documentation=man:syncthing(1)
After=network.target

[Service]
User=pi
ExecStart=/usr/bin/syncthing -no-browser -no-restart -logflags=0
Restart=on-failure
RestartSec=5
SuccessExitStatus=3 4
RestartForceExitStatus=3 4

# Hardening
ProtectSystem=full
PrivateTmp=true
SystemCallArchitectures=native
MemoryDenyWriteExecute=true
NoNewPrivileges=true

[Install]
WantedBy=multi-user.target
EOF

In [ ]:
# start service
sudo systemctl enable syncthing
sudo systemctl start syncthing

## Verify Syncthing is running

In [ ]:
!sudo systemctl --no-pager status syncthing

## Configure from Web Interface

Consult the [Syncthing documentation](https://syncthing.net/) or the [Install Guide](https://pimylifeup.com/raspberry-pi-syncthing/) for details.

Summary of procedure: 

* go to http://yourpi.local:8384
* Click `Danger/Settings`
   * Select `GUI` tab
   * Enter GUI Authentication user and password
      * e.g. your name and something that's hard to guess
   * click Save
* Click `Actions/Show ID`
   * Copy ID to Syncthing on remote (e.g. Mac) you want to share files with
      * Note: Syncthing must be running on remote!
      * on your remote, go to http://localhost:8384
      * `Add Remote Device`
      * On Pi, accept remote request
      * On remote (Mac), the new Pi will show up
      * Click on it and press `Edit`
      * Add the folders you want to share